In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
from numpy.fft import fft, ifft
from scipy.signal import find_peaks
from scipy import signal
from scipy.signal import argrelextrema
py.init_notebook_mode(connected=True)
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#from catboost import CatBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('/content/drive/MyDrive/PCOMP/dataSetPCOMP_time.csv')
df

,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,...,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,device
0,-0.0072,-0.0064,-0.0088,-0.0072,-0.0072,-0.0064,-0.0048,-0.0032,-0.0024,0.0024,...,0.0016,0.0008,-0.0008,-0.0032,-0.0032,-0.0048,-0.0064,-0.0064,-0.0096,fan
1,-8.0000,-8.0000,-8.0000,-0.0104,-0.0072,-0.0064,-0.0064,-0.0048,-0.0032,-0.0024,...,4.0000,0.0024,0.0008,-0.0016,-0.0016,-0.0032,-4.0000,-0.0056,-0.0064,fan
2,-8.0000,-0.0072,-0.0088,-0.0072,-8.0000,-8.0000,-0.0072,-0.0064,-0.0064,-4.0000,...,0.0056,0.0048,0.0016,0.0024,0.0008,0.0000,-0.0016,-0.0032,-0.0048,fan
3,-0.0056,-0.0056,-8.0000,-0.0088,-8.0000,-8.0000,-8.0000,-0.0096,-0.0064,-0.0064,...,0.0088,0.0072,0.0056,0.0048,0.0032,0.0024,-0.0008,-0.0032,-0.0032,fan
4,-4.0000,-0.0056,-0.0064,-0.0064,-8.0000,-0.0088,-0.0088,-8.0000,-8.0000,-0.0072,...,12.0000,0.0096,8.0000,0.0072,0.0088,4.0000,0.0024,0.0008,-0.0008,fan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,...,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,airConditioner
4996,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,936.0000,752.0000,568.0000,352.0000,192.0000,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,airConditioner
4997,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,airConditioner
4998,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,airConditioner


In [3]:
classes = ["microwave", "airConditioner", "refrigerator", "monitor", "fan"]
X = df.drop(columns=['device'])
y = df['device'].apply(classes.index).to_numpy()
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

## Random Forest

In [4]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [RandomForestClassifier(random_state=42)],
        'estimator__n_estimators': [50, 100, 200],        # Número de árvores na floresta
        'estimator__max_depth': [None, 10, 20, 30],      # Profundidade máxima das árvores
        'estimator__min_samples_split': [2, 5, 10],      # Número mínimo de amostras necessárias para dividir um nó
        'estimator__min_samples_leaf': [1, 2, 4]         # Número mínimo de amostras necessárias para ser uma folha
    }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', RandomForestClassifier())])

In [5]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10, verbose=2)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV] END estimator=RandomForestClassifier(random_state=42), estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   1.2s
[CV] END estimator=RandomForestClassifier(random_state=42), estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   0.7s
[CV] END estimator=RandomForestClassifier(random_state=42), estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   0.7s
[CV] END estimator=RandomForestClassifier(random_state=42), estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=RandomForestClassifier(rando

In [6]:
best_model.best_params_

{'estimator': RandomForestClassifier(n_estimators=200, random_state=42),
 'estimator__max_depth': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 200,
 'scaler': StandardScaler()}

In [7]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.9969999812498829
Acurácia:  0.997


## Gradient Boosting

In [8]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [GradientBoostingClassifier(random_state=42)],
        'estimator__n_estimators': [50, 100, 200],        # Número de estimadores (árvores)
        'estimator__learning_rate': [0.01, 0.1, 0.2],    # Taxa de aprendizado
        'estimator__max_depth': [3, 4, 5],               # Profundidade máxima das árvores de decisão
        'estimator__min_samples_split': [2, 3, 4]        # Número mínimo de amostras necessárias para dividir um nó
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', GradientBoostingClassifier())])

In [9]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10, verbose=2)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Fitting 10 folds for each of 81 candidates, totalling 810 fits
[CV] END estimator=GradientBoostingClassifier(random_state=42), estimator__learning_rate=0.01, estimator__max_depth=3, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   4.3s
[CV] END estimator=GradientBoostingClassifier(random_state=42), estimator__learning_rate=0.01, estimator__max_depth=3, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   4.7s
[CV] END estimator=GradientBoostingClassifier(random_state=42), estimator__learning_rate=0.01, estimator__max_depth=3, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   6.1s
[CV] END estimator=GradientBoostingClassifier(random_state=42), estimator__learning_rate=0.01, estimator__max_depth=3, estimator__min_samples_split=2, estimator__n_estimators=50, scaler=StandardScaler(); total time=   4.3s
[CV] END estimator=GradientBoostingClassifier

In [10]:
best_model.best_params_

{'estimator': GradientBoostingClassifier(max_depth=4, n_estimators=200, random_state=42),
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 4,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 200,
 'scaler': StandardScaler()}

In [11]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.9974999570889732
Acurácia:  0.9975


## Decision Tree

In [12]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [DecisionTreeClassifier(random_state=42)],
        'estimator__criterion': ['gini', 'entropy'],    # Critério de divisão
        'estimator__max_depth': [None, 10, 20, 30],     # Profundidade máxima da árvore
        'estimator__min_samples_split': [2, 5, 10],    # Número mínimo de amostras para dividir um nó
        'estimator__min_samples_leaf': [1, 2, 4]       # Número mínimo de amostras em um nó folha

     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', DecisionTreeClassifier())])

In [13]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10, verbose=2)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV] END estimator=DecisionTreeClassifier(random_state=42), estimator__criterion=gini, estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, scaler=StandardScaler(); total time=   0.0s
[CV] END estimator=DecisionTreeClassifier(random_state=42), estimator__criterion=gini, estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, scaler=StandardScaler(); total time=   0.0s
[CV] END estimator=DecisionTreeClassifier(random_state=42), estimator__criterion=gini, estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, scaler=StandardScaler(); total time=   0.0s
[CV] END estimator=DecisionTreeClassifier(random_state=42), estimator__criterion=gini, estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, scaler=StandardScaler(); total time=   0.0s
[CV] END estimator=DecisionTreeClassifier(random_stat

In [ ]:
best_model.best_params_

In [15]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.957099320575346
Acurácia:  0.957
F1-Score:  0.957099320575346
Acurácia:  0.957
